In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [3]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

urls = [
    "https://www.wikipedia.org/wiki/Deep_learning",
    "https://www.wikipedia.org/wiki/Artificial_intelligence",
    "https://www.wikipedia.org/wiki/Machine_learning",
]

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [doc for sublist in docs for doc in sublist]

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=50)
doc_splits = text_splitter.split_documents(docs_list)

# Add all these texts to  the vector store
vector_store = FAISS.from_documents(documents=doc_splits, embedding=OpenAIEmbeddings())

# Create a retriever from the vector store
retriever = vector_store.as_retriever()

In [5]:
retriever.invoke("What is deep learning?")

[Document(id='2ad05190-50df-427c-9a63-69a04540dda7', metadata={'source': 'https://www.wikipedia.org/wiki/Artificial_intelligence', 'title': 'Artificial intelligence - Wikipedia', 'language': 'en'}, page_content='Deep learning'),
 Document(id='49b6b888-640d-41e1-afc3-5b1f0b62e53b', metadata={'source': 'https://www.wikipedia.org/wiki/Machine_learning', 'title': 'Machine learning - Wikipedia', 'language': 'en'}, page_content='Deep learning — branch of ML concerned with artificial neural networks'),
 Document(id='1362305a-f281-41f8-890b-70b75b63df82', metadata={'source': 'https://www.wikipedia.org/wiki/Deep_learning', 'title': 'Deep learning - Wikipedia', 'language': 'en'}, page_content='Fundamentally, deep learning refers to a class of machine learning algorithms in which a hierarchy'),
 Document(id='c35845e9-d243-467c-99e9-16fcda9d1d7a', metadata={'source': 'https://www.wikipedia.org/wiki/Deep_learning', 'title': 'Deep learning - Wikipedia', 'language': 'en'}, page_content='Deep learning

In [6]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever=retriever,
    name="WikipediaRetriever",
    description="Useful for retrieving information from Wikipedia articles about AI, ML, and DL.",
)

In [7]:
retriever_tool

Tool(name='WikipediaRetriever', description='Useful for retrieving information from Wikipedia articles about AI, ML, and DL.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000227B582C180>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000227C4860710>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x00000227B5937D80>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000227C4860710>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={

In [8]:
tools = [retriever_tool]

In [9]:
from typing import Annotated, Sequence
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

In [11]:
def agent(state: AgentState) -> str:
    messages = state["messages"]
    model = ChatGroq(model="qwen-qwq-32b")
    model = model.bind_tools(tools)
    response = model.invoke(messages)
    return {"messages": [response]}

In [12]:
from typing import Annotated, Literal, Sequence
from typing_extensions import TypedDict
from langchain import hub
from langchain_core.messages import HumanMessage, BaseMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import Field, BaseModel

In [ ]:
class Grade(BaseModel):
    """Binary score for relevance check"""
    binary_score: str = Field(
        description="Relevance score 'yes' or 'no'"
    )


def grade_documents(state: AgentState) -> Literal["generate", "rewrite"]:
    print("---------------CHECK RELEVANCE----------------")

    # model
    model = ChatGroq(model="qwen-qwq-32b", streaming=True)

    # llm with tools and validation
    llm_with_tools = model.with_structured_output(Grade)

    # prompt
    prompt = PromptTemplate(
        input_variables=["messages"],
        template="Is the following information relevant to the question? Respond with 'yes' or 'no'.\n\n{messages}",
    )

In [ ]:
from langgraph.graph import END, START, StateGraph
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

# Define a new graph
workflow = StateGraph(AgentState)

# Define the nodes we will cycle through
workflow.add_node("agent", agent)
retrieve = ToolNode([retriever_tool])
workflow.add_node("retrieve", retrieve)
workflow.add_node("rewrite", rewrite)
workflow.add_node("generate", generate)

# Call agent node to decide to retrieve or not
workflow.add_edge(START, "agent")

# Decide whether to retrieve
workflow.add_conditional_edges("agent", tools_condition, {"tools": "retrieve", END: END})
workflow.add_conditional_edges("retrieve", grade_documents)
workflow.add_edge("generate", END)
workflow.add_edge("rewrite", "agent")

# Compile
graph = workflow.compile()